In [7]:
data = pd.read_csv("sample_data/close_0401.csv")
print(data.head())

    Minutes  Stock1  Stock2  Stock3  Stock4  Stock5  Stock6  Stock7  Stock8  \
0  09:31:00   37.00   52.65   23.25   40.35   30.65   71.40   13.10   23.45   
1  09:32:00   36.95   52.60   23.10   40.30   30.95   71.50   13.14   23.50   
2  09:33:00   36.75   52.65   23.20   40.35   30.85   71.65   13.20   23.50   
3  09:34:00   36.60   52.50   23.25   40.25   30.55   71.45   13.16   23.40   
4  09:35:00   36.70   52.30   23.20     NaN   30.55   71.35   13.10   23.35   

   Stock9  ...  Stock91  Stock92  Stock93  Stock94  Stock95  Stock96  Stock97  \
0   16.00  ...    181.5     4.19    22.20      NaN     6.25    53.50     2.76   
1   15.94  ...    184.0     4.17    22.10      NaN     6.34    53.70     2.76   
2     NaN  ...    184.5     4.17    22.25      NaN     6.36    53.90     2.76   
3     NaN  ...    184.3     4.18    22.25      NaN     6.36    53.85     2.76   
4     NaN  ...    183.6     4.17    22.15      NaN     6.36    53.70     2.76   

   Stock98  Stock99  Stock100  
0    7

In [17]:
def ranking(df, window=10, universe_size=100, reversion = 1):
    #Logic of forward filling might need to be re-thought out
    minutes = df.Minutes
    df = df.drop(columns=['Minutes'])
    returns_df = df.pct_change(fill_method='bfill')
    rolling_returns = returns_df.rolling(window).mean()
    valid_returns = rolling_returns[~rolling_returns.isna() & ~np.isinf(rolling_returns)]
    reversion_signals = -reversion *(valid_returns > 0).astype(int)

    reversion_rankings = (reversion_signals.rank(axis=1, ascending=False, method='first') - 50)/universe_size
    return pd.concat([minutes, reversion_rankings], axis=1)

def ranking_transformation(df, power=3):
    minutes = df.Minutes
    df = df.drop(columns=['Minutes'])
    df = df.applymap(lambda x: x**power if power % 2 != 0 else x**power if x >= 0 else -(x**power))
    return pd.concat([minutes, df])

# def normalize_rows(df):
#     minutes = df.Minutes
#     df = df.drop(columns=['Minutes'])
#     positive_sum = df[df > 0].sum(axis=1)
#     negative_sum = df[df < 0].sum(axis=1)
#     positive_sum[positive_sum == 0] = np.nan
#     negative_sum[negative_sum == 0] = np.nan
#     scale_positive = 1 / positive_sum
#     scale_negative = -1 / negative_sum
#     scale_positive = scale_positive.fillna(0)
#     scale_negative = scale_negative.fillna(0)
#     normalized_df = df.apply(lambda row: (row * scale_positive) if row.name in positive_sum.index else (row * scale_negative), axis=1)
#     return pd.concat([minutes, normalized_df], axis=1)


In [32]:
ranked_data

,Minutes,Stock1,Stock2,Stock3,Stock4,Stock5,Stock6,Stock7,Stock8,Stock9,...,Stock91,Stock92,Stock93,Stock94,Stock95,Stock96,Stock97,Stock98,Stock99,Stock100
0,09:31:00,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
1,09:32:00,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
2,09:33:00,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
3,09:34:00,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
4,09:35:00,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,15:56:00,-0.49,-0.21,-0.20,-0.19,-0.18,-0.17,-0.16,-0.15,-0.48,...,0.44,-0.24,0.45,0.46,0.47,0.48,0.49,-0.23,-0.22,0.50
326,15:57:00,-0.49,-0.17,-0.16,-0.48,-0.15,-0.47,-0.14,-0.46,-0.45,...,0.45,-0.21,0.46,0.47,0.48,-0.20,0.49,-0.19,-0.18,0.50
327,15:58:00,-0.49,-0.25,-0.24,-0.23,-0.22,-0.21,-0.20,-0.19,-0.48,...,0.45,-0.29,0.46,0.47,0.48,-0.28,0.49,-0.27,-0.26,0.50
328,15:59:00,-0.49,-0.48,-0.23,-0.22,-0.21,-0.20,-0.19,-0.18,-0.47,...,0.45,0.46,0.47,0.48,-0.27,-0.26,0.49,-0.25,0.50,-0.24


In [11]:
close = pd.read_csv("sample_data/close_0401.csv")
volume = pd.read_csv("sample_data/volume_0401.csv")
ranked_data = ranking(close)




/var/folders/4k/6rq2dc2j4w9fvhjkkjt2x0mr0000gn/T/ipykernel_12079/1897605525.py:5: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_df = df.pct_change(fill_method='bfill')


In [13]:
ranked_data

,Stock1,Stock2,Stock3,Stock4,Stock5,Stock6,Stock7,Stock8,Stock9,Stock10,...,Stock91,Stock92,Stock93,Stock94,Stock95,Stock96,Stock97,Stock98,Stock99,Stock100
0,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,-0.40,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
1,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,-0.40,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
2,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,-0.40,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
3,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,-0.40,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
4,-0.49,-0.48,-0.47,-0.46,-0.45,-0.44,-0.43,-0.42,-0.41,-0.40,...,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,-0.49,-0.21,-0.20,-0.19,-0.18,-0.17,-0.16,-0.15,-0.48,-0.47,...,0.44,-0.24,0.45,0.46,0.47,0.48,0.49,-0.23,-0.22,0.50
326,-0.49,-0.17,-0.16,-0.48,-0.15,-0.47,-0.14,-0.46,-0.45,-0.44,...,0.45,-0.21,0.46,0.47,0.48,-0.20,0.49,-0.19,-0.18,0.50
327,-0.49,-0.25,-0.24,-0.23,-0.22,-0.21,-0.20,-0.19,-0.48,-0.18,...,0.45,-0.29,0.46,0.47,0.48,-0.28,0.49,-0.27,-0.26,0.50
328,-0.49,-0.48,-0.23,-0.22,-0.21,-0.20,-0.19,-0.18,-0.47,-0.17,...,0.45,0.46,0.47,0.48,-0.27,-0.26,0.49,-0.25,0.50,-0.24


In [34]:
def normalize_row(row):
    neg_values = row[row < 0]
    pos_values = row[row >= 0]

    if len(neg_values) > 0:
        neg_sum = neg_values.sum()
        neg_norm = -neg_values / neg_sum
    else:
        neg_norm = pd.Series([0] * len(row))

    if len(pos_values) > 0:
        pos_sum = pos_values.sum()
        pos_norm = pos_values / pos_sum
    else:
        pos_norm = pd.Series([0] * len(row))

    row =np.where(row >= 0, row/pos_sum , -row/neg_sum)
    return row

In [40]:
import pandas as pd
import numpy as np


# ranked_data = ranked_data.drop(columns=['Minutes'])
df_norm = ranked_data.apply(normalize_row, axis=1)
df_norm = pd.DataFrame(df_norm.tolist())
df_norm.columns = ranked_data.columns


# Assuming returns_df is the DataFrame containing returns data
# You can apply the normalization function to it



print("Original DataFrame:")
print(ranked_data)
print("\nNormalized DataFrame:")
print(df_norm)


Original DataFrame:
     Stock1  Stock2  Stock3  Stock4  Stock5  Stock6  Stock7  Stock8  Stock9  \
0     -0.49   -0.48   -0.47   -0.46   -0.45   -0.44   -0.43   -0.42   -0.41   
1     -0.49   -0.48   -0.47   -0.46   -0.45   -0.44   -0.43   -0.42   -0.41   
2     -0.49   -0.48   -0.47   -0.46   -0.45   -0.44   -0.43   -0.42   -0.41   
3     -0.49   -0.48   -0.47   -0.46   -0.45   -0.44   -0.43   -0.42   -0.41   
4     -0.49   -0.48   -0.47   -0.46   -0.45   -0.44   -0.43   -0.42   -0.41   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
325   -0.49   -0.21   -0.20   -0.19   -0.18   -0.17   -0.16   -0.15   -0.48   
326   -0.49   -0.17   -0.16   -0.48   -0.15   -0.47   -0.14   -0.46   -0.45   
327   -0.49   -0.25   -0.24   -0.23   -0.22   -0.21   -0.20   -0.19   -0.48   
328   -0.49   -0.48   -0.23   -0.22   -0.21   -0.20   -0.19   -0.18   -0.47   
329   -0.27   -0.49   -0.26   -0.25   -0.24   -0.23   -0.22   -0.21   -0.20   

     Stock10  ...  Stock91  Sto

In [9]:

from alphas.alpha_baseline_reversion import *
from functions.functions import *
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [10]:
book_value = 1000000
daily_returns = alpha_baseline_reversion(book_value)

Cumulative -44125193.78052534


In [3]:
stats(daily_returns, book_value)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [5]:
from alphas.functions.functions import *
from alphas.alpha3 import *
book_value = 100000
alpha(book_value)

ModuleNotFoundError: No module named 'functions'

In [2]:
from functions.functions import *

ModuleNotFoundError: No module named 'functions'